In [ ]:
# https://huggingface.co/cutelemonlili/Qwen2.5-Math-1.5B-Instruct_MATH_training_response_AIDC-AI__Marco-o1_common_correct_macro_7b
# https://www.kaggle.com/code/liondude/lb-20-qwq-32b-preview-optimized-inference#Inference
# https://huggingface.co/docs/transformers/en/perf_infer_gpu_one
# https://huggingface.co/Qwen/Qwen2.5-Math-1.5B-Instruct
!pip install faiss-gpu sentence-transformers

from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
import torch


import re
import os
import polars as pl
import kaggle_evaluation.aimo_2_inference_server as inference_server

pd.set_option('display.max_colwidth', None)

In [ ]:
SEED = 123

MODEL = "Qwen/Qwen2.5-Math-1.5B-Instruct"
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2" 

TOP_K = 1
THRESHOLD = 0.25

DEBUG = True

In [ ]:
if DEBUG:
    df_reasoning = pd.read_csv('/kaggle/input/reasoning/reasoning.csv')[:-1]
    
    df_problems_text = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv')[:-1]
    
    examples_df = pd.merge(df_problems_text, df_reasoning, on="answer", how="inner")
    
    del df_reasoning, df_problems_text
    
    
else:
    df_reasoning = pd.read_csv('/kaggle/input/reasoning/reasoning.csv')
    
    df_problems_text = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv')
    
    examples_df = pd.merge(df_problems_text, df_reasoning, on="answer", how="inner")
    
    del df_reasoning, df_problems_text
    

#examples_df   

In [ ]:
def load_model():
    model = AutoModelForCausalLM.from_pretrained(MODEL, torch_dtype=torch.float16).to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    return model, tokenizer

def find_similar_examples(current_problem: str, examples_df: pd.DataFrame, similarity_threshold=THRESHOLD):
    if examples_df is None or examples_df.empty:
        return pd.DataFrame()
    
    # Load sentence transformer model
    embedding_model = SentenceTransformer(EMBEDDING_MODEL)
    # Generate embeddings for current problem and all examples
    current_embedding = embedding_model.encode(current_problem.tolist(), convert_to_tensor=False)
    example_embeddings = embedding_model.encode(examples_df['problem'].tolist(), convert_to_tensor=False)
    
    # Normalize embeddings for cosine similarity
    current_embedding = current_embedding.astype(np.float32)
    example_embeddings = example_embeddings.astype(np.float32)
    faiss.normalize_L2(current_embedding)
    faiss.normalize_L2(example_embeddings)
    
    # Build FAISS index
    dimension = example_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner product = cosine similarity for normalized vectors
    index.add(example_embeddings)
    
    # Search for similar examples
    similarities, indices = index.search(current_embedding, TOP_K)

    print('!!!!!!!!!! SIMILARITY SCORES: ', similarities)
    
    # Filter results based on similarity threshold
    mask = similarities[0] > similarity_threshold
    filtered_indices = indices[0][mask]
    
    # Return similar examples
    return examples_df.iloc[filtered_indices]

def create_prompt(problem, similar_examples_df=None):
    system_content  = """You are a very clever math expert who excels at solving complex mathematical problems through careful step-by-step reasoning. When solving problems, you:
1. First read and understand the problem carefully [Think about the key information provided]
2. List all the given information and what needs to be found [Clearly state what we're solving for]
3. Break down the problem into smaller parts [List the steps needed to solve]
4. Solve each part systematically [Show each calculation step]
5. Double-check your work [Verify the solution makes sense]
6. Provide the final answer as an integer [You MUST put the final answer between \\boxed{}]
"""
    
    if similar_examples_df is not None and not similar_examples_df.empty:
        system_content += """IMPORTANT: Here are some similar problems I've solved before. 
        In case you deem it necessary, you can use them as a reference for your approach to solving the given problem.\n\n"""
        
        for _, row in similar_examples_df.iterrows():
            system_content += f"Problem: {row['problem']}\nReasoning: {row['reasoning']}\nAnswer: {row['answer']}\n\n"
        
    user_content = "Now solve this problem step by step:  " + problem.tolist()[0]
    
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    print('!!!!!!!!!! MESSAGGI: ', messages)
    return messages

def extract_answer(response_text: str) -> int:
    # Cerca tutti i numeri all'interno della pattern \boxed{}
    matches = re.findall(r'\\boxed\s*\{(\d+)\}', response_text)
    
    if not matches:
        return 0
    
    # Prende l'ultimo numero trovato
    number = int(matches[-1])
    
    # Se il numero è maggiore di 1000, restituisce il valore modulo 1000
    return number % 1000 if number > 1000 else number

def solve_problem(examples_df: pd.DataFrame, problem: str) -> int:
    # Load model and tokenizer
    model, tokenizer = load_model()

    # Find similar examples
    similar_examples = find_similar_examples(problem, examples_df)
    
    # Create prompt with similar examples only
    messages  = create_prompt(problem, similar_examples)

    chat_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        return_tensors="pt"
    ).to('cuda')
    
    # Generate response
    outputs = model.generate(
        chat_input,
        max_new_tokens=1024 * TOP_K,
        temperature=1e-3,
        repetition_penalty=10.,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print('!!!!!!!!!! LA RISPOSTA è: ', response)
    
    # Extract answer
    solution = extract_answer(response)
    print('!!!!!!!!!! LA SOLUZIONE è: ', solution)
    
    return solution

In [ ]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    
    question = question.item(0)
    answer = solve_problem(examples_df, question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [ ]:

if __name__ == '__main__':
    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.AIMO2InferenceServer(predict).serve()
    else:
        print('Skipping run for now')

In [ ]:
##################### DEBUG #####################
question = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv').tail(1)['problem']
solve_problem(examples_df, question)

In [ ]:
# TODO: 1) Riordinare i file csv 2) Utilizzare modello Instruct e relativa struttura di chat